In [1]:
import os
import torch
import numpy as np

from torch.utils.data import TensorDataset, DataLoader, Subset
import torchvision
from os import path

from src.models import *
from src.methods import *
from src.utils import *
from src.utils import *

data_path = path.join(os.getcwd(), 'project_data/')

USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


In [2]:
X_test = np.load(data_path + "X_test.npy")
y_test = np.load(data_path + "y_test.npy") - 769
person_train_valid = np.load(data_path + "person_train_valid.npy")
X_train_valid = np.load(data_path + "X_train_valid.npy")
y_train_valid = np.load(data_path + "y_train_valid.npy") - 769
person_test = np.load(data_path + "person_test.npy")

In [3]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [4]:
X_train_valid = X_train_valid[:, np.newaxis, :, :]
X_test = X_test[:, np.newaxis, :, :]

In [5]:
X_train_valid = torch.from_numpy(X_train_valid).float()
y_train_valid = torch.from_numpy(y_train_valid).long()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

train_valid_dataset = TensorDataset(X_train_valid, y_train_valid)
test_dataset = TensorDataset(X_test, y_test)

# Splitting the dataset into train and valid sets
num_train = int(0.8 * len(train_valid_dataset))
num_valid = len(train_valid_dataset) - num_train
train_indices, valid_indices = random_split(range(len(train_valid_dataset)), [num_train, num_valid])

train_dataset = Subset(train_valid_dataset, train_indices)
valid_dataset = Subset(train_valid_dataset, valid_indices)

# Wrapping datasets with GaussianNoisyDataset
train_dataset_noisy = GaussianNoisyDataset(train_dataset, mean=0., std=1.)
valid_dataset_noisy = GaussianNoisyDataset(valid_dataset, mean=0., std=1.)
test_dataset_noisy = GaussianNoisyDataset(test_dataset, mean=0., std=1.)

batch_size = 32
train_loader = DataLoader(train_dataset_noisy, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset_noisy, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset_noisy, batch_size=batch_size, shuffle=False)

print('Training/Valid data shape:', X_train_valid.shape)
print('Test data shape:', X_test.shape)

Training/Valid data shape: torch.Size([2115, 1, 22, 1000])
Test data shape: torch.Size([443, 1, 22, 1000])


In [6]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-6, weight_decay=0.0005)

train(model, train_loader, valid_loader, criterion, optimizer, device=device, epochs=50)

test_accuracy = evaluate(model, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

 20%|██        | 10/50 [00:21<01:26,  2.15s/it]

Epoch [10/50], Loss: 0.4634, Validation Accuracy: 52.25%


 40%|████      | 20/50 [00:44<01:09,  2.32s/it]

Epoch [20/50], Loss: 0.2118, Validation Accuracy: 51.06%


 48%|████▊     | 24/50 [00:55<00:59,  2.30s/it]

Early stopping!
Test Accuracy: 50.56%


In [6]:
lstm = LSTMModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-6, weight_decay=0.0005)

lstm = lstm.to(device)
train(lstm, train_loader, valid_loader, criterion, optimizer, device=device)

test_accuracy = evaluate(lstm, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

 50%|█████     | 10/20 [02:30<02:32, 15.29s/it]

Epoch [10/20], Loss: 1.2707, Validation Accuracy: 27.90%


100%|██████████| 20/20 [04:42<00:00, 14.12s/it]

Epoch [20/20], Loss: 1.0690, Validation Accuracy: 28.84%


Test Accuracy: 26.19%


In [6]:
cnn_lstm = CNN_LSTM()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_lstm.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-6)

cnn_lstm = cnn_lstm.to(device)
train(cnn_lstm, train_loader, valid_loader, criterion, optimizer, device=device, epochs=50)

test_accuracy = evaluate(cnn_lstm, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

/home/qimingliu0831/anaconda3/envs/nlp/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
 20%|██        | 10/50 [00:06<00:27,  1.45it/s]

Epoch [10/50], Loss: 1.2336, Validation Accuracy: 39.01%


 40%|████      | 20/50 [00:16<00:33,  1.10s/it]

Epoch [20/50], Loss: 1.0758, Validation Accuracy: 47.75%


 60%|██████    | 30/50 [00:24<00:14,  1.37it/s]

Epoch [30/50], Loss: 0.9771, Validation Accuracy: 52.72%


 80%|████████  | 40/50 [00:33<00:07,  1.40it/s]

Epoch [40/50], Loss: 0.8258, Validation Accuracy: 52.72%


100%|██████████| 50/50 [00:41<00:00,  1.20it/s]

Epoch [50/50], Loss: 0.7014, Validation Accuracy: 52.72%
Test Accuracy: 53.95%


In [9]:
ult_cnn = UltimateConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ult_cnn.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-6, weight_decay=0.0005)

ult_cnn = ult_cnn.to(device)
train(ult_cnn, train_loader, valid_loader, criterion, optimizer, device=device, epochs=100)

test_accuracy = evaluate(ult_cnn, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

  0%|          | 0/100 [00:00<?, ?it/s]

 10%|█         | 10/100 [00:20<03:04,  2.05s/it]

Epoch [10/100], Loss: 0.8852, Validation Accuracy: 57.21%


 20%|██        | 20/100 [00:41<02:44,  2.06s/it]

Epoch [20/100], Loss: 0.7216, Validation Accuracy: 61.47%


...

Epoch [70/100], Loss: 0.4030, Validation Accuracy: 66.90%


 80%|████████  | 80/100 [02:44<00:41,  2.08s/it]

Epoch [80/100], Loss: 0.3565, Validation Accuracy: 65.01%


 87%|████████▋ | 87/100 [03:00<00:27,  2.08s/it]

Early stopping!
Test Accuracy: 66.37%
